A linear equation is defined as:

$$ y(x_{0},x{1},\ldots,x{n}) = m_{0} x_{0} + m_{1} x_{1} + \ldots + m_{n} x_{n} + b $$

Loss function:

$$ L(m,b) = \frac{1}{2N} \sum_{i=1}^N (b + m x_{i} - y_{i})^2 $$

Let

$$ f(m,b) = b + m x_{i} - y_{i}  $$

and

$$ g(x) = \frac{1}{2N} \sum_{i=1}^N x^2 $$

So

$$ \frac{\partial}{\partial x} g(x) = \frac{1}{N} \sum_{i=1}^N x $$

$$ \frac{\partial}{\partial b} f(m,b) = 1 $$

$$ \frac{\partial}{\partial m} f(m,b) = x_{i} $$

$$ \frac{\partial}{\partial b} g(f(m,b)) = \frac{\partial}{\partial b} g(b + m x_{i} - y_{i}) \frac{\partial}{\partial b} f(m,b) $$

$$ \frac{\partial}{\partial b} L(m,b) = \frac{1}{N} \sum_{i=1}^N (b + m x_{i} - y_{i}) $$

$$ \frac{\partial}{\partial m} L(m,b) = \frac{1}{N} \sum_{i=1}^N (b + m x_{i} - y_{i}) x_{i} $$


In [3]:
import numpy as np

Loss function:

$$ L(m,b) = \frac{1}{2N} \sum_{i=1}^N (b + m x_{i} - y_{i})^2 $$

In [12]:
def loss(m, b, points):
    sum = 0.0
    n = len(points)
    for i in range(0, n):
        x = points[i, 0]
        y = points[i, 1]
        sum += (b + m * x - y) ** 2
    return sum / (2.0 * float(n))

In [11]:
points = np.genfromtxt("linear_regression_data.csv", delimiter=",")
[ x for x in range(0, 10) ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]